# Most Well-Connected Areas in the US (in 2018)

This is a tutorial on how to find the most well-connected regions of the U.S. via air travel.

The U.S. Bureau of Transportation Statistics provides data on monthly air travel from all certificated U.S. air carriers. The data can be found [here](https://www.transtats.bts.gov/DL_SelectFields.asp?Table_ID=258). 

We utilized this data to determine which areas in the U.S. are most well-connected.

We’ll first investigate which region is travelled through the most and then which regions share the most travellers.

We chose 2018 data to avoid any impacts COVID-19 might’ve had on travel.

# Data Preprocessing

Let's first look at the data.

First, we'll need to import some libraries. 

In [1]:
import metagraph as mg
import pandas as pd
from collections import Counter
from IPython.display import display

Let's see what the data looks like.

In [2]:
RAW_DATA_CSV = './raw_data.csv' # https://www.transtats.bts.gov/DL_SelectFields.asp?Table_ID=258
raw_data_df = pd.read_csv(RAW_DATA_CSV)
raw_data_df.head()

PASSENGERS  FREIGHT  MAIL  DISTANCE UNIQUE_CARRIER  AIRLINE_ID  \
0         0.0    410.0   0.0     616.0             WN     19393.0   
1         0.0    184.0   0.0    2592.0             WN     19393.0   
2         0.0     87.0   0.0    2445.0             WN     19393.0   
3         0.0     10.0   0.0     432.0             WN     19393.0   
4         0.0    100.0   0.0     129.0             WN     19393.0   

      UNIQUE_CARRIER_NAME UNIQUE_CARRIER_ENTITY REGION CARRIER  ...  \
0  Southwest Airlines Co.                 06725      D      WN  ...   
1  Southwest Airlines Co.                 06725      D      WN  ...   
2  Southwest Airlines Co.                 06725      D      WN  ...   
3  Southwest Airlines Co.                 06725      D      WN  ...   
4  Southwest Airlines Co.                 06725      D      WN  ...   

  DEST_STATE_ABR  DEST_STATE_FIPS  DEST_STATE_NM  DEST_WAC  YEAR  QUARTER  \
0             TN               47      Tennessee        54  2018        2   
1             CA                6     California        91  2018        2   
2             NY               36       New York        22  2018        2   
3             AR                5       Arkansas        71  2018        2   
4             OR               41         Oregon        92  2018        2   

  MONTH DISTANCE_GROUP CLASS  Unnamed: 36  
0     6              2     F          NaN  
1     6              6     F          NaN  
2     6              5     F          NaN  
3     6              1     F          NaN  
4     6              1     F          NaN  

[5 rows x 37 columns]

A city market is a region that an airport supports. For example, New York City has many airports (and it’s sometimes cheaper to fly into and out of different airports), but all of their airports serve the same region / city market.

Since we’re mostly concerned with where passengers will end up going (and not which airport they choose), we’ll view city markets as the regions that we’re trying to determine the connectedness of.

There looks to be a lot of information that’s not relevant to finding the most well connected region, e.g. the distance of a flight path (we’re concerned with connectedness in the sense that many people commonly travel between the two areas). Let’s filter those out.

Let’s also filter out any flight paths with zero passengers (these flights are usually flights transporting packages).

In [3]:
RELEVANT_COLUMNS = [
    'PASSENGERS',
    'ORIGIN_AIRPORT_ID', 'ORIGIN_AIRPORT_SEQ_ID', 'ORIGIN_CITY_MARKET_ID', 'ORIGIN', 'ORIGIN_CITY_NAME', 'ORIGIN_STATE_ABR', 'ORIGIN_STATE_NM',
    'DEST_AIRPORT_ID',   'DEST_AIRPORT_SEQ_ID',   'DEST_CITY_MARKET_ID',   'DEST',   'DEST_CITY_NAME',   'DEST_STATE_ABR',   'DEST_STATE_NM',
]
relevant_df = raw_data_df[RELEVANT_COLUMNS]
relevant_df = relevant_df[relevant_df.PASSENGERS != 0.0]
relevant_df.head()

PASSENGERS  ORIGIN_AIRPORT_ID  ORIGIN_AIRPORT_SEQ_ID  \
44447         1.0              12523                1252306   
44448         1.0              12523                1252306   
44449         1.0              12610                1261001   
44450         1.0              11298                1129806   
44451         1.0              15991                1599102   

       ORIGIN_CITY_MARKET_ID ORIGIN       ORIGIN_CITY_NAME ORIGIN_STATE_ABR  \
44447                  32523    JNU             Juneau, AK               AK   
44448                  32523    JNU             Juneau, AK               AK   
44449                  32610    KAE               Kake, AK               AK   
44450                  30194    DFW  Dallas/Fort Worth, TX               TX   
44451                  35991    YAK            Yakutat, AK               AK   

      ORIGIN_STATE_NM  DEST_AIRPORT_ID  DEST_AIRPORT_SEQ_ID  \
44447          Alaska            11545              1154501   
44448          Alaska            11619              1161902   
44449          Alaska            10204              1020401   
44450           Texas            11292              1129202   
44451          Alaska            14828              1482805   

       DEST_CITY_MARKET_ID DEST       DEST_CITY_NAME DEST_STATE_ABR  \
44447                31545  ELV       Elfin Cove, AK             AK   
44448                31619  EXI  Excursion Inlet, AK             AK   
44449                30204  AGN           Angoon, AK             AK   
44450                30325  DEN           Denver, CO             CO   
44451                34828  SIT            Sitka, AK             AK   

      DEST_STATE_NM  
44447        Alaska  
44448        Alaska  
44449        Alaska  
44450      Colorado  
44451        Alaska

We’ll want to have our data in an edge list format.

We'll want the city markets to be the nodes. 

Since we're using betweenness centrality to determine connectedness, we'll want paths with less total weight to be the ones denoting paths with more passengers. More elegant metrics might be considered in practice, but we'll use `1/number_of_passengers` for the weights for our example.

We'll create an edge list with such weights using pandas.

In [4]:
passenger_flow_df = relevant_df[['ORIGIN_CITY_MARKET_ID', 'DEST_CITY_MARKET_ID', 'PASSENGERS']]
passenger_flow_df = passenger_flow_df.groupby(['ORIGIN_CITY_MARKET_ID', 'DEST_CITY_MARKET_ID']) \
                        .PASSENGERS.sum() \
                        .reset_index()
passenger_flow_df['INVERSE_PASSENGER_COUNT'] = passenger_flow_df.PASSENGERS.map(lambda passenger_count: 1/passenger_count)
assert len(passenger_flow_df[passenger_flow_df.INVERSE_PASSENGER_COUNT != passenger_flow_df.INVERSE_PASSENGER_COUNT]) == 0, "Edge list has NaN weights."
passenger_flow_df.head()

ORIGIN_CITY_MARKET_ID  DEST_CITY_MARKET_ID  PASSENGERS  \
0                  30005                30349         4.0   
1                  30005                31214        10.0   
2                  30005                31517       193.0   
3                  30005                35731         7.0   
4                  30006                30056         5.0   

   INVERSE_PASSENGER_COUNT  
0                 0.250000  
1                 0.100000  
2                 0.005181  
3                 0.142857  
4                 0.200000

Since the data has city market IDs and don’t have names because an airport can serve regions containing multiple cities, it’d be useful to get a mapping from city market IDs to city names and airports.

In [5]:
origin_city_market_id_info_df = relevant_df[['ORIGIN_CITY_MARKET_ID', 'ORIGIN', 'ORIGIN_CITY_NAME']] \
                                    .rename(columns={'ORIGIN_CITY_MARKET_ID': 'CITY_MARKET_ID',
                                                     'ORIGIN': 'AIRPORT',
                                                     'ORIGIN_CITY_NAME': 'CITY_NAME'})
dest_city_market_id_info_df = relevant_df[['DEST_CITY_MARKET_ID', 'DEST', 'DEST_CITY_NAME']] \
                                    .rename(columns={'DEST_CITY_MARKET_ID': 'CITY_MARKET_ID',
                                                     'DEST': 'AIRPORT',
                                                     'DEST_CITY_NAME': 'CITY_NAME'})
city_market_id_info_df = pd.concat([origin_city_market_id_info_df, dest_city_market_id_info_df])
city_market_id_info_df = city_market_id_info_df.groupby('CITY_MARKET_ID').agg({'AIRPORT': set, 'CITY_NAME': set})
city_market_id_info_df.head()

AIRPORT           CITY_NAME
CITY_MARKET_ID                                
30005                {05A}  {Little Squaw, AK}
30006                {06A}      {Kizhuyak, AK}
30007                {KLW}       {Klawock, AK}
30009           {HOM, 09A}         {Homer, AK}
30010                {1B1}        {Hudson, NY}

# Visualize Routes

Let's visualize the routes just to get an idea of what the data looks like. 

The data from the U.S. Bureau of Transportation Statistics doesn't contain the latitudinal or longitudinal information regarding airports or city markets, so we'll have to first grab and integrate that data from elsewhere. We'll use the Global Airport Database.

In [6]:
# https://www.partow.net/miscellaneous/airportdatabase/
airports_df = pd.read_csv('./GlobalAirportDatabase.txt', sep=':', header=None, names=['ICAO_Code', 'IATA_Code', 'Airport_Name', 'City_Town', 'Country', 'Latitude_Degrees', 'Latitude_Minutes', 'Latitude_Seconds', 'Latitude_Direction', 'Longitude_Degrees', 'Longitude_Minutes', 'Longitude_Seconds', 'Longitude_Direction', 'Altitude', 'Latitude_Decimal_Degrees', 'Longitude_Decimal_Degrees'])

city_market_id_info_df_with_geodata = \
                city_market_id_info_df.apply(lambda x: pd.Series(list(x.AIRPORT)), axis=1) \
                    .stack() \
                    .reset_index(level=1, drop=True) \
                    .to_frame('AIRPORT') \
                    .reset_index() \
                    .merge(airports_df[['IATA_Code', 'Latitude_Decimal_Degrees', 'Longitude_Decimal_Degrees']], 
                           left_on='AIRPORT', right_on='IATA_Code') \
                    .drop(['IATA_Code'], axis=1) \
                    .groupby('CITY_MARKET_ID').mean() \
                    .reset_index().set_index('CITY_MARKET_ID') \
                    .join(city_market_id_info_df[['AIRPORT', 'CITY_NAME']])

passenger_flow_df_with_geodata = passenger_flow_df \
                                    .merge(city_market_id_info_df_with_geodata[['Latitude_Decimal_Degrees','Longitude_Decimal_Degrees']].rename(columns={'Latitude_Decimal_Degrees':'ORIG_LAT', 'Longitude_Decimal_Degrees':'ORIG_LONG'}), 
                                        right_on='CITY_MARKET_ID', left_on='ORIGIN_CITY_MARKET_ID') \
                                    .merge(city_market_id_info_df_with_geodata[['Latitude_Decimal_Degrees','Longitude_Decimal_Degrees']].rename(columns={'Latitude_Decimal_Degrees':'DEST_LAT', 'Longitude_Decimal_Degrees':'DEST_LONG'}),
                                        right_on='CITY_MARKET_ID', left_on='DEST_CITY_MARKET_ID') 

In [7]:
city_market_id_info_df_with_geodata.head()

Latitude_Decimal_Degrees  Longitude_Decimal_Degrees  \
CITY_MARKET_ID                                                        
30009                             59.646                   -151.476   
30070                             57.750                   -152.494   
30073                             70.195                   -148.465   
30082                             55.206                   -162.724   
30107                             71.285                   -156.766   

                   AIRPORT        CITY_NAME  
CITY_MARKET_ID                               
30009           {HOM, 09A}      {Homer, AK}  
30070           {ADQ, KDK}     {Kodiak, AK}  
30073                {SCC}  {Deadhorse, AK}  
30082                {CDB}   {Cold Bay, AK}  
30107                {BRW}     {Barrow, AK}

In [8]:
passenger_flow_df_with_geodata.head()

ORIGIN_CITY_MARKET_ID  DEST_CITY_MARKET_ID  PASSENGERS  \
0                  30009                30009       466.0   
1                  30070                30009       170.0   
2                  30073                30009         1.0   
3                  30299                30009     33027.0   
4                  31555                30009        11.0   

   INVERSE_PASSENGER_COUNT  ORIG_LAT  ORIG_LONG  DEST_LAT  DEST_LONG  
0                 0.002146    59.646   -151.476    59.646   -151.476  
1                 0.005882    57.750   -152.494    59.646   -151.476  
2                 1.000000    70.195   -148.465    59.646   -151.476  
3                 0.000030    61.194   -149.921    59.646   -151.476  
4                 0.090909    60.573   -151.245    59.646   -151.476

In [9]:
from pyproj import Transformer
from bokeh.tile_providers import get_provider, Vendors
from bokeh.plotting import figure, show
from bokeh.io import output_notebook, push_notebook
from typing import Tuple
import warnings

TRANSFORMER = Transformer.from_crs(4326, 3857)
def lat_long_to_mercator(lat: float, long: float) -> Tuple[float, float]:
    mercator_x, mercator_y = TRANSFORMER.transform(lat, long)
    return mercator_x, mercator_y

fig = figure(plot_width=1600, 
             plot_height=800,
             x_range=(-11_000_000, 5_000_000),
             y_range=(-5_000_000, 12_000_000),
             x_axis_type='mercator', 
             y_axis_type='mercator')

tile_provider = get_provider(Vendors.ESRI_IMAGERY)    
fig.add_tile(tile_provider)

output_notebook()
figure_display_handle = show(fig, notebook_handle=True)

mercator_df = passenger_flow_df_with_geodata[['ORIG_LAT', 'ORIG_LONG', 'DEST_LAT', 'DEST_LONG']] \
                .apply(lambda row: lat_long_to_mercator(row[0], row[1])+lat_long_to_mercator(row[2], row[3]), axis=1)
passenger_flow_df_with_geodata['ORIG_MERCATOR_X'] = mercator_df.map(lambda x: x[0])
passenger_flow_df_with_geodata['ORIG_MERCATOR_Y'] = mercator_df.map(lambda x: x[1])
passenger_flow_df_with_geodata['DEST_MERCATOR_X'] = mercator_df.map(lambda x: x[2])
passenger_flow_df_with_geodata['DEST_MERCATOR_Y'] = mercator_df.map(lambda x: x[3])

xs = list(zip(passenger_flow_df_with_geodata.ORIG_MERCATOR_X, passenger_flow_df_with_geodata.DEST_MERCATOR_X))
ys = list(zip(passenger_flow_df_with_geodata.ORIG_MERCATOR_Y, passenger_flow_df_with_geodata.DEST_MERCATOR_Y))
fig.multi_line(xs, ys,
         line_width=2, line_alpha=0.1, line_color='orange')
fig.circle(passenger_flow_df_with_geodata.ORIG_MERCATOR_X, passenger_flow_df_with_geodata.ORIG_MERCATOR_Y, size=5, color='red')
fig.circle(passenger_flow_df_with_geodata.DEST_MERCATOR_X, passenger_flow_df_with_geodata.DEST_MERCATOR_Y, size=5, color='red')
push_notebook(handle=figure_display_handle)

Loading BokehJS ...

# Which region is travelled through the most?

We’re going to determine which region is travelled through the most using Betweenness Centrality as it measures exactly that. There are a variety of algorithms to choose from, but we’ll stick to using solely Betweenness Centrality for this tutorial.

We’ll first create a metagraph graph for the data.

In [10]:
r = mg.resolver
passenger_flow_graph_wrapped = r.wrappers.EdgeMap.PandasEdgeMap(passenger_flow_df, 
                                                                'ORIGIN_CITY_MARKET_ID', 
                                                                'DEST_CITY_MARKET_ID', 
                                                                'INVERSE_PASSENGER_COUNT',
                                                                is_directed=True)

Note that we use the inverse passenger count as the weights to ensure that the shortest paths are the paths that have the most passengers.

Let's calculate the Betweenness Centrality. 

In [11]:
%time betweenness_centrality = r.algos.vertex_ranking.betweenness_centrality(passenger_flow_graph_wrapped, 100, False, False)

CPU times: user 1.72 s, sys: 119 µs, total: 1.72 s
Wall time: 1.71 s


Let’s look at the results and find the highest scores (which would give us the city market IDs that are most travelled through).

In [12]:
type(betweenness_centrality)

metagraph.plugins.python.types.PythonNodeMap

In [13]:
number_of_best_scores = 15
best_betweenness_centrality_scores = sorted(betweenness_centrality.value.items(), key=lambda x: x[1], reverse=True)[:number_of_best_scores]
best_betweenness_centrality_scores

[(32575, 49405.0),
 (30559, 44352.0),
 (31703, 32659.0),
 (30299, 31495.0),
 (30977, 21128.0),
 (30397, 17202.0),
 (30194, 12061.0),
 (30325, 10482.0),
 (31517, 9537.0),
 (32457, 6194.0),
 (30113, 5833.0),
 (32467, 5503.0),
 (30154, 5140.0),
 (32134, 4412.0),
 (30070, 4141.0)]

Now that we have the city market IDs with the best scores, let’s find out which regions those city market IDs correspond to using the mapping from city market IDs to city names and airports we made earlier.

In [14]:
best_betweenness_centrality_scores_df = pd.DataFrame(best_betweenness_centrality_scores).rename(columns={0:'CITY_MARKET_ID', 1:'BETWEENNESS_CENTRALITY_SCORE'}).set_index('CITY_MARKET_ID')
best_betweenness_centrality_scores_df.join(city_market_id_info_df).sort_values('BETWEENNESS_CENTRALITY_SCORE', ascending=False)

BETWEENNESS_CENTRALITY_SCORE  \
CITY_MARKET_ID                                 
32575                                49405.0   
30559                                44352.0   
31703                                32659.0   
30299                                31495.0   
30977                                21128.0   
30397                                17202.0   
30194                                12061.0   
30325                                10482.0   
31517                                 9537.0   
32457                                 6194.0   
30113                                 5833.0   
32467                                 5503.0   
30154                                 5140.0   
32134                                 4412.0   
30070                                 4141.0   

                                                      AIRPORT  \
CITY_MARKET_ID                                                  
32575                {SMO, BUR, LGB, HHR, LAX, ONT, VNY, SNA}   
30559                                    {LKE, BFI, KEH, SEA}   
31703           {JFK, LGA, JRB, HPN, JRA, ISP, EWR, SWF, TSS}   
30299                                         {MRI, DQL, ANC}   
30977                          {LOT, MDW, DPA, PWK, GYY, ORD}   
30397                                    {FTY, QMA, PDK, ATL}   
30194                     {DAL, AFW, FTW, RBD, FWH, DFW, ADS}   
30325                                              {APA, DEN}   
31517                               {FBK, A01, EIL, MTX, FAI}   
32457                                    {CCR, SFO, OAK, SJC}   
30113                                                   {BET}   
32467                          {MIA, OPF, TMB, MPB, FLL, FXE}   
30154                                                   {ACK}   
32134                                              {HIK, HNL}   
30070                                              {ADQ, KDK}   

                                                        CITY_NAME  
CITY_MARKET_ID                                                     
32575           {Long Beach, CA, Hawthorne, CA, Van Nuys, CA, ...  
30559                                  {Seattle, WA, Kenmore, WA}  
31703           {Islip, NY, White Plains, NY, Newark, NJ, Newb...  
30299                                             {Anchorage, AK}  
30977             {Gary, IN, Chicago/Romeoville, IL, Chicago, IL}  
30397                                 {Atlanta, GA, Kennesaw, GA}  
30194           {Dallas, TX, Dallas/Fort Worth, TX, Fort Worth...  
30325                                                {Denver, CO}  
31517               {Fairbanks, AK, Fairbanks/Ft. Wainwright, AK}  
32457           {Concord, CA, San Francisco, CA, Oakland, CA, ...  
30113                                                {Bethel, AK}  
32467                            {Miami, FL, Fort Lauderdale, FL}  
30154                                             {Nantucket, MA}  
32134                                              {Honolulu, HI}  
30070                                                {Kodiak, AK}

This is what we'd expect. Highly populated areas like New York City or Los Angeles are the most traveled through areas.

However, it's suprising that Anchorage is more travelled through than a hub like Dallas!

There’s a good explanation for Anchorage being a very travelled through region: Since Alaska is so sparsely populated, a well-connected road infrtastructure was never built. Thus, to travel between cities in Alaska, air travel is the only option. More information can be found [here](https://en.wikipedia.org/wiki/List_of_airports_in_Alaska).